<a href="https://colab.research.google.com/github/dotmanjohn/kosh/blob/master/Preparation_of_Speed_Dating_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparing the Speed Dating Dataset (Data Cleaning)
As an entrepreneur, you are planning to launch a new dating app into the market. The key feature that will differentiate your app from other competitors will be your high-performing user-matching algorithm. Before building this model, you have partnered with a speed dating company to collect data from real events. **You just received the dataset from your partner company but realized it is not as clean as you expected; there are missing and incorrect values. Your task is to fix the main data quality issues in this dataset**.

In [ ]:
import pandas as pd
file_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter11/dataset/Speed_Dating_Data.csv'

In [ ]:
df = pd.read_csv(file_url)

In [ ]:
df.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,...,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,1,11.0,0,0.14,0,27.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,0,6.0,8.0,8.0,8.0,8.0,6.0,7.0,4.0,2.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,...,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,2,12.0,0,0.54,0,22.0,2.0,60.0,0.0,0.0,40.0,0.0,0.0,0,7.0,8.0,10.0,7.0,7.0,5.0,8.0,4.0,2.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,...,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,3,13.0,1,0.16,1,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,1,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,...,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,4,14.0,1,0.61,0,23.0,2.0,30.0,5.0,15.0,40.0,5.0,5.0,1,7.0,8.0,9.0,8.0,9.0,8.0,7.0,7.0,2.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,...,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,5,15.0,1,0.21,0,24.0,3.0,30.0,10.0,20.0,10.0,10.0,20.0,1,8.0,7.0,9.0,6.0,9.0,7.0,8.0,6.0,2.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,...,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(8378, 195)

In [ ]:
# Print out the number of duplicate rows
df.duplicated().sum()

0

In [ ]:
# Print out the number of duplicate rows, but this time only look at the identifier columns
df.loc[df.duplicated(), ['iid', 'id', 'partner', 'pid']].sum()

iid        0.0
id         0.0
partner    0.0
pid        0.0
dtype: float64

Looking at the dataset description document, we know that the values of the following variables should range between 1 and 10: 'imprace', 'imprelig', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'exphappy', and 'satis_2'. In the next few steps, we are going to check that there are no unexpected values in these columns.

Create a variable called scale_1_10. This will list the following column names: 'imprace', 'imprelig', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'exphappy', and 'satis_2':

In [ ]:
scale_1_10 = ['imprace', 'imprelig', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
              'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'exphappy', 'satis_2']

Create a function called check_range that takes a column (a pandas series), a minimum value, and a maximum value as input parameters. The function will check whether each row of the given column is outside the given range (below the minimum value or above the maximum value) and return the corresponding list of binary values:

In [ ]:
def check_range(column, min_value, max_value):
  return (column < min_value) | (column >max_value)

Test the function on the imprace column with 1 and 10 as the minimum and maximum values, respectively. Then, save the output into a variable called unexpected_mask and print the sum to check how many cases are outside this range:

In [ ]:
unexpected_mask = check_range(df['imprace'], 1, 10)
unexpected_mask.sum()

8

Define a function called print_unexpected that takes a DataFrame, a column name, and a list of binary values as input parameters. This function will check whether the sum of the binary values is over 0. If it is, the case prints out the column name, this sum, and the unique values of the given columns and the rows that match the binary list (keeping only True values) using the pandas .loc and .unique() methods:

In [ ]:
def print_unexpected(df, col_name, unexpected_mask):
  if unexpected_mask.sum() > 0:
    print(col_name)
    print(unexpected_mask.sum())
    print(df.loc[unexpected_mask,col_name].unique())

In [ ]:
# Test your function on the 'imprace' column using the output of the previous function, that is, unexpected_mask:
print_unexpected(df, 'imprace', unexpected_mask)

imprace
8
[0.]


Create a function called check_ranges that takes a DataFrame, a list of columns, and minimum and maximum values as input parameters. This function will iterate through each column from the given column list, call the check_range function, and pass its output to the print_unexpected function, which was earlier defined

In [ ]:
def check_ranges(df, col_list, min_value, max_value):
  for col_name in col_list:
    unexpected_mask = check_range(df[col_name], min_value, max_value)
    print_unexpected(df, col_name, unexpected_mask)

In [ ]:
# Test this function with the dataset and the scale_1_10 variables you defined in step 9 , and use 1 and 10 as their minimum and maximum values, respectively:
check_ranges(df, scale_1_10, 1, 10)

imprace
8
[0.]
museums
18
[0.]
art
18
[0.]
hiking
18
[0.]
gaming
137
[14.  0.]
clubbing
18
[0.]
reading
51
[13.]
theater
18
[0.]
movies
18
[0.]
concerts
18
[0.]
yoga
36
[0.]


Create a function called replace_value that takes a DataFrame, a column name,an incorrect value, and a new value as input parameters. This function will subset all the rows that are equal to the incorrect value for the given column and replace it with the new given value. Then, it will print out the column's name and the list of unique values for this column:

In [ ]:
def replace_value(df, col_name, incorrect_value, new_value):
  df.loc[df[col_name] == incorrect_value, col_name] = new_value
  print(col_name)
  print(df[col_name].unique())

In [ ]:
# Test the replace_value function on the gaming column, where 14 is the incorrect value and 10 is the new value:
replace_value(df, 'gaming', 14, 10)

gaming
[ 1.  5.  4.  6.  2.  3.  7.  8. 10. nan  9.  0.]


In [ ]:
# Use the replace_value function on the reading column, where 13 is the incorrect value and 10 is the new value:
replace_value(df, 'reading', 13, 10)

reading
[ 6. 10.  7.  9.  8.  4.  5. nan  2.  3.  1.]


Create a for loop that will iterate through the following suffixes: ['1_1', '1_2', '1_3', '1_s', '2_1', '2_2', '2_3', '4_1', '4_2', '4_3', '7_2', and '7_3']. For each of them, create a list comprehension (or another for loop) so that we can extract the columns that contain the given suffix by using the .endswith() method and store them into a variable called suffix_cols. Then, apply the check_ranges function to this list and use 0 and 100 as their minimum and maximum values, respectively:

In [ ]:
for suffix in ['1_1', '1_2', '1_3', '1_s', '2_1', '2_2', '2_3', '4_1', '4_2', '4_3', '7_2', '7_3']:
  suffix_cols = [col for col in df.columns if col.endswith(suffix)]
  check_ranges(df, suffix_cols, 0, 100)

No output is displayed, which means that all these columns have values within the expected range, that is, between 0 and 100.

Create a for loop that's similar to what we created above for the following suffixes, where 1 and 10 are the minimum and maximum values, respectively: ['3_1', '3_2', '3_3', '5_1', '5_2', '5_3', '3_s']:

In [ ]:
for suffix in ['3_1', '3_2', '3_3', '5_1', '5_2', '5_3', '3_s']:
  suffix_cols = [col for col in df.columns if col.endswith(suffix)]
  check_ranges(df, suffix_cols, 1, 10)

attr3_3
112
[12.]
sinc3_3
173
[12.]
intel3_3
233
[12.]
fun3_3
153
[12.]
amb3_3
147
[12.]


As we can see, all the columns ending with 3_3 have 12 as their unexpected values. Let's say that, after consultation with the surveyors, we agree to replace these values with 10.

Create a for loop that iterates through the list of columns ending with 3_3 and call the replace_values function for each of them. Provide 12 as the incorrect value and 10 as the new value:

In [ ]:
for col_name in ['attr3_3', 'sinc3_3', 'intel3_3', 'fun3_3', 'amb3_3']:
  replace_value(df, col_name, 12, 10)

attr3_3
[ 5.  7. nan  6.  4.  9.  8.  3. 10.  2.]
sinc3_3
[ 7.  6. nan  5.  8.  9. 10.  4.  3.  2.]
intel3_3
[ 7.  9. nan  6. 10.  8.  5.  4.  3.]
fun3_3
[ 7.  9. nan  8.  6.  3.  5. 10.  2.  4.]
amb3_3
[ 7.  4. nan  5. 10.  9.  8.  6.  2.  3.  1.]


We have fixed the unexpected values for these columns.

In [ ]:
# Print the data type of each variable using the dtypes attribute:
df.dtypes

iid           int64
id          float64
gender        int64
idg           int64
condtn        int64
             ...   
attr5_3     float64
sinc5_3     float64
intel5_3    float64
fun5_3      float64
amb5_3      float64
Length: 195, dtype: object

As we can see, most of the columns have been detected as numerical variables, but by looking at the dataset description document, we know that most of them are categorical. So we change their data types.

Create a list called num_cols that contains the following list of columns: 'round', 'order', 'int_corr', 'age', 'mn_sat', 'income', and 'expnum'

In [ ]:
num_cols = ['round', 'order', 'int_corr', 'age', 'mn_sat', 'income', 'expnum']

Create another list called cat_cols that contains the remaining column names (excluding the ones in num_cols) of this DataFrame using the attribute columns combined with the .difference() method:

In [ ]:
cat_cols = df.columns.difference(num_cols)

Create a for loop that will iterate through cat_cols and change the data type for each column into a category by using the .astype() method:

In [ ]:
for col_name in cat_cols:
  df[col_name] = df[col_name].astype('category')

In [ ]:
# Print the data type of each variable using the dtypes attribute:
df.dtypes

iid         category
id          category
gender      category
idg         category
condtn      category
              ...   
attr5_3     category
sinc5_3     category
intel5_3    category
fun5_3      category
amb5_3      category
Length: 195, dtype: object

We have sorted out the data types for each column. Now, let's see whether we have any missing columns in the numerical fields.

In [ ]:
# Print the number of missing values for each column in num_cols by combining the .isna() and .sum() methods:
df[num_cols].isna().sum()

round          0
order          0
int_corr     158
age           95
mn_sat      5245
income      4099
expnum      6578
dtype: int64

In [ ]:
# Print the unique values of the 'int_corr' variable using the .unique() method:
df['int_corr'].unique()

array([ 0.14,  0.54,  0.16,  0.61,  0.21,  0.25,  0.34,  0.5 ,  0.28,
       -0.36,  0.29,  0.18,  0.1 , -0.21,  0.32,  0.73,  0.6 ,  0.07,
        0.11,  0.39, -0.24, -0.14,  0.09, -0.04, -0.3 , -0.26, -0.15,
       -0.47, -0.18,  0.05,  0.37,  0.35,  0.15, -0.19, -0.43,  0.  ,
       -0.17,  0.08, -0.16,  0.06, -0.05, -0.13, -0.06,  0.33, -0.51,
        0.12,  0.19,  0.47,  0.03,  0.46,  0.43,  0.52, -0.46, -0.27,
        0.59,  0.31, -0.34, -0.03, -0.11,  0.42, -0.4 , -0.23,  0.17,
        0.68, -0.01, -0.35,  0.3 ,  0.65,  0.24,  0.41,  0.49,  0.01,
        0.22, -0.08,  0.27,  0.44,  0.62, -0.2 , -0.02, -0.33, -0.52,
       -0.1 ,  0.58, -0.57, -0.31, -0.07, -0.32,  0.04, -0.12,  0.48,
       -0.22, -0.29,  0.38,  0.53, -0.38,  0.02, -0.28,  0.13,  0.2 ,
         nan, -0.41, -0.44,  0.51, -0.48,  0.4 ,  0.26,  0.77, -0.49,
       -0.25, -0.09,  0.45, -0.39,  0.83,  0.57, -0.61,  0.72, -0.37,
        0.23, -0.58,  0.8 , -0.56,  0.63, -0.63,  0.71,  0.36,  0.56,
        0.55,  0.76,

The values of the int_corr column range between -1 and 1. It seems like they have been normalized. Since there are no extreme values or outliers, we can impute the missing values with the mean of this variable. This is what we are going to do in the next few steps.

Create a condition mask called int_corr_mask that finds the missing values in the 'int_corr' column by using the .isna() method

In [ ]:
int_corr_mask = df['int_corr'].isna()

In [ ]:
# Display the number of missing values for this column by using the .sum() method on int_corr_mask:
int_corr_mask.sum()

158

Extract the mean of int_corr using the .mean() method and store it in a new variable called int_corr_mean. Print out its value

In [ ]:
int_corr_mean = df['int_corr'].mean()
print(int_corr_mean)

0.19600973236009664


The average value for this column is 0.196. We need to replace all the missing values with this value in the int_corr column.

Replace all the missing values in the int_corr variable with their averages by using the .fillna() method along with the inplace=True parameter

In [ ]:
df['int_corr'].fillna(int_corr_mean, inplace=True)

In [ ]:
# Print the number of missing values for int_corr by combining the .isna() and .sum() methods:
df['int_corr'].isna().sum()

0

Create a new variable called missing_num_cols that contains the following columns: age, mn_sat, income, and expnum

In [ ]:
missing_num_cols = ['age', 'mn_sat', 'income', 'expnum']

Create a for loop that will iterate through the columns in missing_num_cols and print out their names and a list of their unique values using the .unique() method

In [ ]:
for col_name in missing_num_cols:
  print(col_name)
  print(df[col_name].unique())

age
[21. 24. 25. 23. 22. 26. 27. 30. 28. nan 29. 34. 35. 32. 39. 20. 19. 18.
 37. 33. 36. 31. 42. 38. 55.]
mn_sat
[  nan 1070. 1258. 1400. 1290. 1460. 1430. 1215. 1330. 1450. 1155. 1140.
 1360. 1402. 1250. 1210. 1220. 1410. 1260. 1380. 1030. 1309. 1308. 1050.
 1100. 1310. 1490. 1188. 1097. 1212. 1340. 1034. 1185. 1242. 1160. 1099.
 1214. 1270. 1110. 1178. 1060. 1157. 1180. 1014. 1341.  990. 1320. 1159.
 1370. 1105. 1365. 1011. 1130. 1206. 1331. 1191.  914. 1200. 1080. 1090.
 1092. 1470. 1149. 1134. 1230. 1267. 1280. 1227. 1239.]
income
[ 69487.  65929.     nan  37754.  86340.  60304.  54620.  48652.  29237.
  56580.  36782.  38548.  52010.  28418.  43185.  23152.  43664.  48441.
  61152.  36485.  41507.  17134.  30038.  33772.  24997.  42096.  28891.
  62635.  12063.  29809.  26482.  30147.  39919.  41466.  23988.  28989.
  50948.  38022.  47559.  53539.  32159.  53940.  40753.  38207.  46166.
  30973.  28317.  26645.  25589.  55223. 109031.  40409.  21597.  76624.
  35968.  51725.  55

The values for these columns haven't been normalized and some of them have outliers. This time, we are going to need to use their medians to fill in the missing values.

Create a for loop, calculate the median of each column and save it into a variable called col_median. Then, impute the missing values with this median value by using the .fillna() method along with the inplace=True parameter and print the name of the column and its median value:

In [ ]:
for col_name in missing_num_cols:
  col_median = df[col_name].median()
  df[col_name].fillna(col_median, inplace=True)
  print(col_name)
  print(col_median)

age
26.0
mn_sat
1310.0
income
43185.0
expnum
4.0


Create a for loop, print the name of each column and its number of missing values by combining the .isna() and .sum() methods

In [ ]:
for col_name in missing_num_cols:
  print(col_name)
  print(df[col_name].isna().sum())

age
0
mn_sat
0
income
0
expnum
0


We have cleaned up most of the main quality issues in this dataset. We looked for duplication, incorrect values, incorrect data types, and missing values. We are now more confident in using this modified version of the dataset to build a matching algorithm if we really were to work on this project.